In [220]:
import pandas as pd
import numpy as np

In [221]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
np.seterr(all='raise')

{'divide': 'raise', 'over': 'raise', 'under': 'raise', 'invalid': 'raise'}

In [222]:
# load the materialized benchmark config
stats_file = "../results/combined.stats.csv"
sim_df = pd.read_csv(stats_file, header=0)
assert((sim_df["input_mode"] == "serial").sum() > 0)
sim_df.head(5)

,cycles,instructions,num_blocks,warp_inst,exec_time_sec,is_release_build,dram_reads,dram_writes,dram_accesses,l2_reads,l2_writes,l2_accesses,l2_read_hits,l2_write_hits,l2_read_misses,l2_write_misses,l2_hits,l2_misses,input_dtype,input_length,input_mode,input_threads,input_run_ahead,target,benchmark,input_id,l2_read_hit_rate,l2_write_hit_rate,l2_read_miss_rate,l2_write_miss_rate
0,1117.0,10516.0,1.0,0.0,0.044,True,56,13,69.0,10,4,14.0,0.0,0.0,10.0,4.0,0.0,14.0,32,100,serial,4.0,5.0,Simulate,vectorAdd,0,NaN,NaN,NaN,NaN
1,1117.0,10516.0,1.0,0.0,0.054,True,56,13,69.0,10,4,14.0,0.0,0.0,10.0,4.0,0.0,14.0,32,100,serial,4.0,5.0,Simulate,vectorAdd,0,NaN,NaN,NaN,NaN
2,1117.0,10516.0,1.0,0.0,0.070,True,56,13,69.0,10,4,14.0,0.0,0.0,10.0,4.0,0.0,14.0,32,100,serial,4.0,5.0,Simulate,vectorAdd,0,NaN,NaN,NaN,NaN
3,1117.0,10516.0,1.0,0.0,0.051,True,56,13,69.0,10,4,14.0,0.0,0.0,10.0,4.0,0.0,14.0,32,100,serial,4.0,5.0,Simulate,vectorAdd,0,NaN,NaN,NaN,NaN
4,1117.0,10516.0,1.0,0.0,0.150,True,56,13,69.0,10,4,14.0,0.0,0.0,10.0,4.0,0.0,14.0,32,100,deterministic,4.0,5.0,Simulate,vectorAdd,1,NaN,NaN,NaN,NaN


In [94]:
# ideas for stats

# overall correlation per target a) with native b) with accelsim
# all targets: bar plots with the per benchmark and target
# table: error when using parallel execution, and speedup

In [95]:
stat_cols = [
    "exec_time_sec",
    "cycles",
    "num_blocks",
    "instructions",
    "warp_inst",
    "dram_reads",
    "dram_writes",
    "l2_accesses",
    "l2_read_hit_rate",
    "l2_write_hit_rate",
    "l2_read_miss_rate",
    "l2_write_miss_rate",
    "l2_read_hits",
    "l2_write_hits",
    "l2_read_misses",
    "l2_write_misses",
    "l2_hits",
    "l2_misses",
]

index_cols = ["target", "benchmark", "input_id"]

In [96]:
# mean stddev, rel err, mae, rmse for native profiling per metric
benchmarks = ["vectorAdd"]

selected_df = sim_df.copy()
selected_df = selected_df[selected_df["target"] == "Profile"]
if len(benchmarks) > 0:
    selected_df = selected_df[selected_df["benchmark"].isin(benchmarks)]


# assert((selected_df["is_release_build"] == True).all())
# print(selected_df[stat_cols])

# target	benchmark
# [["benchmark", "target", "cycles", "kernel_name"]]
grouped = selected_df.groupby(index_cols)
# grouped[["cycles", "instructions"]].std()
grouped[stat_cols].std()
# selected_df

exec_time_sec       cycles  num_blocks  \
target  benchmark input_id                                           
Profile vectorAdd 0              0.139839   698.205062         0.0   
                  1              0.179013  2841.638484         0.0   
                  2              0.083948  1020.049725         0.0   
                  3              0.062288   686.155253         0.0   
                  4              0.125004   721.782593         0.0   
                  5              0.859590  1478.956008         0.0   

                            instructions  warp_inst  dram_reads  dram_writes  \
target  benchmark input_id                                                     
Profile vectorAdd 0                  0.0        0.0  181.876759     7.306010   
                  1                  0.0        0.0  312.638165    18.590619   
                  2                  0.0        0.0  487.289658   181.137425   
                  3                  0.0        0.0   92.034052    15.317383   
                  4                  0.0        0.0  858.194358    92.222195   
                  5                  0.0        0.0  147.468867   349.254142   

                            l2_accesses  l2_read_hit_rate  l2_write_hit_rate  \
target  benchmark input_id                                                     
Profile vectorAdd 0          235.945286          0.000000                0.0   
                  1          242.693588          0.316228                0.0   
                  2          276.221248          0.000000                0.0   
                  3          127.037002          0.000000                0.0   
                  4          198.508914          0.000000                0.0   
                  5          208.932631          0.000000                0.0   

                            l2_read_miss_rate  l2_write_miss_rate  \
target  benchmark input_id                                          
Profile vectorAdd 0                  0.000000                 0.0   
                  1                  0.316228                 0.0   
                  2                  0.000000                 0.0   
                  3                  0.000000                 0.0   
                  4                  0.000000                 0.0   
                  5                  0.000000                 0.0   

                            l2_read_hits  l2_write_hits  l2_read_misses  \
target  benchmark input_id                                                
Profile vectorAdd 0             0.000000            0.0      236.524746   
                  1           265.315096            0.0      253.311161   
                  2             0.000000            0.0      273.948981   
                  3             0.000000            0.0      118.810774   
                  4             0.000000            0.0      190.945047   
                  5             0.000000            0.0      203.085204   

                            l2_write_misses  l2_hits      l2_misses  
target  benchmark input_id                                           
Profile vectorAdd 0               13.063945      0.0   10375.039268  
                  1               13.492385      0.0   43843.259646  
                  2               14.009521      0.0  357727.093067  
                  3               10.119289      0.0    9387.528237  
                  4               17.281975      0.0   61996.862602  
                  5               16.949598      0.0  552722.893905

In [212]:
# error table for metrics
benchmarks = ["vectorAdd"]
# benchmarks = ["matrixmul"]
# targets = ["Simulate"]
selected_df = sim_df.copy()
if len(benchmarks) > 0:
    selected_df = selected_df[selected_df["benchmark"].isin(benchmarks)]
# only compare serial gpucachesim
# selected_df = selected_df[selected_df["input_mode"] != "nondeterministic"]
selected_df = selected_df[(selected_df["input_mode"] == "serial") ^ (selected_df["input_mode"].isnull())]
# selected_df = selected_df[selected_df["target"].isin(targets)]]
# print(selected_df[selected_df["target"] == "Simulate"])
# print(selected_df["target"].unique().tolist())

# assert((selected_df["is_release_build"] == True).all())

# target	benchmark
# [["benchmark", "target", "cycles", "kernel_name"]]
# input_cols = [c for c in selected_df.columns if c.startswith('input_') and c != "input_id" and c != "input_mode"]
# print(input_cols)
input_cols = ["input_dtype", "input_length"]
# + input_cols input_cols
# values = ["cycles", "instructions", "l2_accesses", "l2_hits", "l2_misses", "dram_accesses", "dram_reads", "dram writes"]
#  
test = selected_df.groupby(index_cols, dropna=False)[stat_cols + input_cols].mean().reset_index()
# test = test.drop(columns="input_id")
# print(test)

test = test.pivot(index=["benchmark"] + input_cols, columns='target', values=stat_cols)
# test = test.pivot(index=["benchmark"], columns='target', values=['cycles', "instructions"])
# test = test.pivot_table(index = ["benchmark"] + [ 'input_dtype', 'input_length' ], columns = "target", values = ['cycles', "instructions"]) # , aggfunc = len)
test.T.head(n=100)

benchmark                                vectorAdd                            \
input_dtype                                   32.0                             
input_length                               100.0       1000.0        10000.0   
                   target                                                      
exec_time_sec      AccelsimSimulate        0.00000      0.0000  0.000000e+00   
                   PlaygroundSimulate      0.00000      0.0000  0.000000e+00   
                   Profile                 3.14550      3.1008  3.328100e+00   
                   Simulate                0.05475      0.0805  5.772500e-01   
cycles             AccelsimSimulate     1285.00000   1648.0000  2.139000e+03   
                   PlaygroundSimulate   1117.00000   1228.0000  1.310000e+03   
                   Profile              6450.15000   8262.7000  7.161700e+03   
                   Simulate             1117.00000   1228.0000  1.311000e+03   
num_blocks         AccelsimSimulate        1.00000      1.0000  1.000000e+01   
                   PlaygroundSimulate      1.00000      1.0000  1.000000e+01   
                   Profile                 0.00000      0.0000  0.000000e+00   
                   Simulate                1.00000      1.0000  1.000000e+01   
instructions       AccelsimSimulate    10516.00000  22216.0000  2.221600e+05   
                   PlaygroundSimulate  10516.00000  22216.0000  2.221600e+05   
                   Profile             10516.00000  22216.0000  2.221600e+05   
                   Simulate            10516.00000  22216.0000  2.221600e+05   
warp_inst          AccelsimSimulate       10.75000     23.0000  2.269375e+01   
                   PlaygroundSimulate     10.75000     23.0000  2.269375e+01   
                   Profile                10.75000     23.0000  2.269375e+01   
                   Simulate                0.00000      0.0000  0.000000e+00   
dram_reads         AccelsimSimulate       56.00000    392.0000  3.764000e+03   
                   PlaygroundSimulate     56.00000    392.0000  7.256000e+03   
                   Profile               222.60000    496.2000  2.831900e+03   
                   Simulate               56.00000    392.0000  5.244000e+03   
dram_writes        AccelsimSimulate       13.00000    125.0000  1.250000e+03   
                   PlaygroundSimulate     13.00000    125.0000  1.250000e+03   
                   Profile                16.60000     25.5000  1.349000e+02   
                   Simulate               13.00000    125.0000  1.250000e+03   
l2_accesses        AccelsimSimulate       14.00000     98.0000  9.410000e+02   
                   PlaygroundSimulate     14.00000     98.0000  9.410000e+02   
                   Profile               452.20000    649.2000  4.154200e+03   
                   Simulate               14.00000     98.0000  9.410000e+02   
l2_read_hit_rate   AccelsimSimulate            NaN         NaN           NaN   
                   PlaygroundSimulate          NaN         NaN           NaN   
                   Profile                 0.00000      0.1000  0.000000e+00   
                   Simulate                    NaN         NaN           NaN   
l2_write_hit_rate  AccelsimSimulate            NaN         NaN           NaN   
                   PlaygroundSimulate          NaN         NaN           NaN   
                   Profile                 0.00000      0.0000  0.000000e+00   
                   Simulate                    NaN         NaN           NaN   
l2_read_miss_rate  AccelsimSimulate            NaN         NaN           NaN   
                   PlaygroundSimulate          NaN         NaN           NaN   
                   Profile                 1.00000      0.9000  1.000000e+00   
                   Simulate                    NaN         NaN           NaN   
l2_write_miss_rate AccelsimSimulate            NaN         NaN           NaN   
                   PlaygroundSimulate          NaN         NaN           NaN   


In [98]:
# bar plots for cycles

benchmarks = ["vectorAdd"]
targets = ["Simulate"]
selected_df = sim_df.copy()
selected_df = selected_df[selected_df["benchmark"].isin(benchmarks)]
selected_df = selected_df[selected_df["target"].isin(targets)]

assert((selected_df["is_release_build"] == True).all())

In [223]:
# relative error for nondeterministic gpucachesim
benchmarks = ["vectorAdd"]
targets = ["Simulate"]
selected_df = sim_df.copy()
selected_df = selected_df[selected_df["benchmark"].isin(benchmarks)]
selected_df = selected_df[selected_df["target"].isin(targets)]

assert((selected_df["is_release_build"] == True).all())

serial = selected_df[selected_df["input_mode"] == "serial"]
nondet = selected_df[selected_df["input_mode"] == "nondeterministic"]
print((nondet["input_id"] == 3).sum())
print(nondet[nondet["input_id"] == 41][["cycles", "l2_reads", "dram_accesses"]])
print(len(serial))
print(len(nondet))

# those are fully distinct
print(serial["input_id"].unique())
print(nondet["input_id"].unique())

# print(serial[[c for c in serial.columns if c.startswith('input_')]])
# print(nondet[[c for c in nondet.columns if c.startswith('input_')]])
bench_cols = ["benchmark"]
input_cols = ["input_dtype", "input_length", "input_memory_only", "input_cores_per_cluster"]
# selected_df[["benchmark", "target", "cycles"]]

# join based on input_cols
joined = nondet.merge(serial, on=bench_cols + input_cols, how="left", suffixes=('_nondet', '_serial'))
# selected_df
# nondet

joined = joined[joined["input_id_nondet"] == 41]
# print(joined["cycles_nondet"] - joined["cycles_serial"])
joined[[
    "benchmark", "input_dtype", "input_length", "input_memory_only", "input_cores_per_cluster" "input_id_nondet",  "input_id_serial",
    "input_threads_nondet", "input_run_ahead_nondet",
    "exec_time_sec_nondet", "exec_time_sec_serial",
    "cycles_nondet", "cycles_serial",
    "l2_accesses_nondet", "l2_accesses_serial"
]].T
# (joined["cycles_nondet"] - joined["cycles_serial"]).abs() / joined["cycles_serial"]

4
     cycles  l2_reads  dram_accesses
164  1600.0      1252        14440.0
165  1600.0      1252        14440.0
166  1600.0      1252        14440.0
167  1600.0      1252        14440.0
24
96
[ 0  7 14 21 28 35]
[ 3  4  5  6 10 11 12 13 17 18 19 20 24 25 26 27 31 32 33 34 38 39 40 41]


KeyError: 'input_memory_only'

In [100]:
# gpusims.plot.metrics.Cycles: plot_scatter,
fig = metrics_plot_func(
    data=per_config_metric_df,
    config=config,
    metric_cls=metric_cls,
    title=f"{metric_cls.name} Correlation ({config.name})",
)
# filename = ["scatter", metric_cls.name, config.key]
# filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
# filename = filename.with_suffix(".pdf")
# fig.write_image(filename, **PDF_OPTS)
fig.show()

NameError: name 'metrics_plot_func' is not defined

In [101]:
for metric_cls, metrics_plot_func in metrics.items():
    print(metric_cls.name)
    all_metric_df = []
    # for (config_name, config) in selected_configs_copy.items():
    for config_name in selected_configs_copy:
        config = configs[config_name]
        per_config_metric_df = []
        
        # for (bench_name, bench) in selected_benchmarks_copy.items():
        for bench_name, selected_bench_inputs in selected_benchmarks_copy:
            bench = benchmarks[bench_name]
            supported_simulators = [
                sim_name for sim_name in selected_simulators
                if bench.enabled(sim_name) # and inp.enabled(sim_name)
            ]
            # for inp in bench.inputs:
            for inp_args, inp_abbr in selected_bench_inputs:
                # print(config_name, bench_name, inp)
                
                inp = next(i for i in bench.inputs if i.args.strip() == inp_args.strip())
                assert inp is not None, f"input {inp_args} does not exist"
                
                plot_data = PlotData(benchmark=bench, config=config, inp=inp)
                # for (sim_name, sim) in selected_simulators_copy.items():
                for sim_name in supported_simulators:
                    sim = simulators[sim_name]
                    
                    if not bench.enabled(sim_name):
                        continue
                    if not inp.enabled(sim_name):
                        continue
                    # print(sim_name, config_name, bench_name)
                    bench_config = sim(
                        run_dir=run_dir / sim_name.lower(),
                        benchmark=bench,
                        config=config,
                    )
                    if not bench_config.input_path(inp).is_dir():
                        print(f"WARN: {bench_config.input_path(inp)} does not exist")
                        continue

                    plot_data[sim_name] = bench_config.load_dataframe(inp)

                metric = metric_cls(plot_data)
                metric_df = metric.compute()
                metric_df["Benchmark"] = f"{bench.name}<br>{inp.args}"
                metric_df["Config"] = config_name

                per_config_metric_df.append(metric_df)
                all_metric_df.append(metric_df)

        per_config_metric_df = pd.concat(per_config_metric_df)
        # break
        # continue
        
        fig = metrics_plot_func(
            data=per_config_metric_df,
            config=config,
            metric_cls=metric_cls,
            title=f"{metric_cls.name} Correlation ({config.name})",
        )
        filename = ["scatter", metric_cls.name, config.key]
        filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
        filename = filename.with_suffix(".pdf")
        fig.write_image(filename, **PDF_OPTS)
        print("wrote", filename)
    
    all_metric_df = pd.concat(all_metric_df)
    if True:
        fig = plot_scatter_subplots(
            in_data=all_metric_df,
            metric_cls=metric_cls,
            title=f"{metric_cls.name} Correlation",
        )
        filename = ["per-configs-scatter", metric_cls.name]
        filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
        filename = filename.with_suffix(".pdf")
        fig.write_image(filename, **PDF_OPTS)
        print("wrote", filename)
        
    fig = metrics_plot_func(
        data=all_metric_df,
        config=config,
        metric_cls=metric_cls,
        title=f"{metric_cls.name} Correlation",
    )
    filename = ["all-configs-scatter", metric_cls.name]
    filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
    filename = filename.with_suffix(".pdf")
    fig.write_image(filename, **PDF_OPTS)
    print("wrote", filename)

# all_metric_df

NameError: name 'metrics' is not defined